# Planner → Executor → Critic Agent Demo

This notebook demonstrates a serious agentic workflow:

1. **Planner**: Breaks a high-level mission into concrete steps.
2. **Executor**: Runs each step using tool-augmented reasoning.
   - Tools: Wikipedia REST summary, OpenWeather live weather, corporate_hotel policy.
3. **Critic**: Audits each step's output for quality, factual completeness, executive relevance.
4. **Synthesizer**: Produces a final executive brief.

This is bigger than ReAct and Reflection alone.
Here, the agent is acting like an operations lead: plan → gather → self-check → brief.

In [ ]:
import os, re, requests, textwrap
from typing import Dict, Callable, List
from openai import OpenAI

# Set keys before running:
# os.environ['OPENAI_API_KEY'] = 'sk-...'
# os.environ['OPENWEATHER_API_KEY'] = 'your-openweather-key'

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
OPENWEATHER_API_KEY = os.environ.get('OPENWEATHER_API_KEY', '<PUT_YOUR_KEY_HERE>')
WIKI_HEADERS = {'User-Agent': 'planner-executor-critic-notebook/1.0'}

def wikipedia_summary(topic: str) -> str:
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    r = requests.get(url, headers=WIKI_HEADERS, timeout=10)
    if r.status_code != 200:
        return f"[wiki] error {r.status_code}: {r.text[:200]}"
    d = r.json()
    return f"{d.get('title','')} — {d.get('description','')}\n{d.get('extract','')}"

def weather_brief(latlon: str) -> str:
    if OPENWEATHER_API_KEY.startswith('<PUT_'):
        return '[weather] Please set OPENWEATHER_API_KEY.'
    lat, lon = [float(x.strip()) for x in latlon.split(',')]
    url = 'https://api.openweathermap.org/data/2.5/weather'
    params = {'lat': lat, 'lon': lon, 'appid': OPENWEATHER_API_KEY}
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return f"[weather] error {r.status_code}: {r.text[:200]}"
    d = r.json()
    k2c = lambda k: round(k - 273.15, 1)
    desc = d['weather'][0]['description']
    temp_c = k2c(d['main']['temp'])
    return f"{desc}, {temp_c}°C"

def corporate_hotel(city: str) -> str:
    if city.lower() == 'hyderabad':
        return 'MetroLink Executive Suites (~₹5400/night). Walkable to Hitech City offices. Breakfast+gym included. Policy-approved.'
    return 'No approved hotel found for this city.'

TOOLS: Dict[str, Callable[[str], str]] = {
    'wikipedia_summary': wikipedia_summary,
    'weather_brief': weather_brief,
    'corporate_hotel': corporate_hotel,
}

def call_model(system_prompt: str, user_prompt: str) -> str:
    resp = client.responses.create(
        model='o4-mini',
        input=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt},
        ],
    )
    if hasattr(resp, 'output_text'):
        return resp.output_text
    try:
        return resp.choices[0].message.content
    except Exception:
        if hasattr(resp, 'output'):
            if isinstance(resp.output, list):
                return '\n'.join(str(x) for x in resp.output)
            return str(resp.output)
        return str(resp)

SYSTEM_PLANNER = """
You are a planning agent.
Break the high-level GOAL into a short numbered plan (3-6 steps).
Each step must be concrete, observable, and something we can attempt with tools.

Your output format MUST be:

PLAN:
1. ...
2. ...
3. ...
4. ...
"""

def make_plan(goal: str):
    plan_text = call_model(
        SYSTEM_PLANNER,
        f"GOAL:\n{goal}\n\nCreate the PLAN now."
    )
    steps = []
    in_plan = False
    for line in plan_text.splitlines():
        line=line.strip()
        if line.upper().startswith('PLAN'):
            in_plan = True
            continue
        if in_plan:
            m = re.match(r'^\d+\.\s*(.+)$', line)
            if m:
                steps.append(m.group(1).strip())
    return steps

SYSTEM_EXECUTOR = """
You are an execution agent with tool access.

TOOLS you MAY REQUEST:
1. wikipedia_summary(topic: str)
2. weather_brief(lat,lon as string "LAT,LON")
3. corporate_hotel(city: str)

Protocol:
- Use Thought: ... to reason.
- If you need a tool:
  Action: <tool_name>
  Action Input: <argument>
- After I run that tool I will respond with Observation: <tool_result>
- Continue until you can answer this step.
- When done with THIS STEP ONLY, return:
  Final Answer: <concise result for this step>

Rules:
- Stay focused ONLY on the given step description.
"""

ACTION_RE = re.compile(r"Action:\s*(?P<tool>\w+)\s*[\r\n]+Action Input:\s*(?P<input>.+)", re.I)
FINAL_RE = re.compile(r"Final Answer:\s*(?P<final>.*)", re.I|re.S)

def interpret_executor_output(text: str):
    m_final = FINAL_RE.search(text)
    if m_final:
        return {'type':'final','answer':m_final.group('final').strip(),'raw':text}
    m_act = ACTION_RE.search(text)
    if m_act:
        return {
            'type':'action',
            'tool':m_act.group('tool').strip(),
            'input':m_act.group('input').strip(),
            'raw':text,
        }
    return {'type':'final','answer':text.strip(),'raw':text}

def execute_step(step_desc: str):
    trace = ''
    step_result = None
    for _ in range(5):
        llm_out = call_model(
            SYSTEM_EXECUTOR,
            f"STEP DESCRIPTION:\n{step_desc}\n\nTRACE SO FAR:\n{trace}\n\nFollow the protocol."
        )
        decision = interpret_executor_output(llm_out)
        trace += llm_out + '\n'
        if decision['type'] == 'final':
            step_result = decision['answer']
            break
        tool_fn = TOOLS.get(decision['tool'])
        if tool_fn is None:
            obs = f"[ERROR] Unknown tool '{decision['tool']}'"
        else:
            obs = tool_fn(decision['input'])
        trace += f'Observation: {obs}\n'
    return {'trace': trace, 'result': step_result}

SYSTEM_CRITIC = """
You are a critical reviewer.
Given the STEP DESCRIPTION and the STEP RESULT,
identify any gaps, vagueness, factual risk, compliance/policy risk,
or missing executive relevance. Then rewrite an improved version.

Your output format MUST be:

CRITIQUE:
- bullet points of issues...

REVISED STEP RESULT:
<improved version>
"""

def critique_step(step_desc: str, step_result: str):
    critic_out = call_model(
        SYSTEM_CRITIC,
        f"STEP DESCRIPTION:\n{step_desc}\n\nSTEP RESULT:\n{step_result}\n\nNow critique and revise."
    )
    parts = critic_out.split('REVISED STEP RESULT:')
    critique_text = parts[0].strip()
    revised = parts[1].strip() if len(parts) > 1 else step_result
    return {'critique': critique_text, 'revised': revised}

SYSTEM_SYNTHESIZER = """
You are an executive briefing agent.
Given all REVISED STEP RESULTS, write a short executive brief.
Audience: VP-level. Keep it focused on signal and action.

Format:
EXECUTIVE BRIEF:
<final bullet points or short paragraphs>
"""

def synthesize_final(goal: str, revised_steps: List[str]):
    joined = '\n\n'.join(f'- {txt}' for txt in revised_steps if txt and txt.strip())
    final_out = call_model(
        SYSTEM_SYNTHESIZER,
        f"USER GOAL:\n{goal}\n\nREVISED STEP RESULTS:\n{joined}\n\nNow write the EXECUTIVE BRIEF."
    )
    return final_out


### Run the full pipeline
We:
1. Generate a PLAN.
2. EXECUTE each step (with tools) and capture its trace.
3. CRITIQUE each step result to upgrade quality.
4. SYNTHESIZE a final executive brief.

In [ ]:
goal = (
    'Brief me for a Hyderabad business trip: what is Hyderabad, what is the current weather (17.44,78.38), '
    'and which approved hotel should I book near Hitech City?'
)
coords = '17.44,78.38'

# 1. Plan
plan_steps = make_plan(goal)
# heuristic: inject coords hint into any weather-related steps
plan_steps = [
    (s + f' Use coordinates {coords} for weather.' if 'weather' in s.lower() else s)
    for s in plan_steps
]
print('===== PLAN =====')
for i, step_desc in enumerate(plan_steps, start=1):
    print(f"{i}. {step_desc}")

# 2. Execute each step
executed = []
for step_desc in plan_steps:
    out = execute_step(step_desc)
    executed.append((step_desc, out))

print('\n===== STEP EXECUTION TRACES =====')
for i, (desc, info) in enumerate(executed, start=1):
    print(f"\n--- Step {i}: {desc} ---")
    print(info['trace'])
    print('Step Result:', info['result'])

# 3. Critique each step
revised_step_results = []
print('\n===== STEP CRITIQUES =====')
for i, (desc, info) in enumerate(executed, start=1):
    crit = critique_step(desc, info['result'] or '')
    revised_step_results.append(crit['revised'])
    print(f"\nCritique for Step {i}: {desc}")
    print(crit['critique'])
    print('\nRevised Step Result:')
    print(crit['revised'])

# 4. Final synthesis
final_brief = synthesize_final(goal, revised_step_results)
print('\n===== FINAL EXECUTIVE BRIEF =====')
print(final_brief)


### Why this pattern matters
- **Planner phase** shows the agent can behave like a project lead.
- **Executor phase** shows tool grounding, not hallucination.
- **Critic phase** shows built-in QA and policy awareness step-by-step.
- **Synthesizer phase** shows final communication tailored to a VP.

This is the closest form to "autonomous analyst" you can responsibly ship:
It plans, it gathers evidence from approved sources, it self-audits, and it produces an executive brief you can defend.